In [1]:
%pip install opencv-python
%pip install mediapipe
!python.exe -m pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
%pip uninstall mediapipe -y
%pip install mediapipe-nightly

Found existing installation: mediapipe 0.10.30
Uninstalling mediapipe-0.10.30:
  Successfully uninstalled mediapipe-0.10.30
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.


   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   -- ------------------------------------- 0.8/10.5 MB 6.7 MB/s eta 0:00:02
   ------------ --------------------------- 3.4/10.5 MB 11.2 MB/s eta 0:00:01
   ----------------------- ---------------- 6.3/10.5 MB 11.7 MB/s eta 0:00:01
   ---------------------------- ----------- 7.6/10.5 MB 10.0 MB/s eta 0:00:01
   ------------------------------- -------- 8.4/10.5 MB 10.6 MB/s eta 0:00:01
   ------------------------------- -------- 8.4/10.5 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------  10.5/10.5 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------- 10.5/10.5 MB 7.5 MB/s  0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install scikit-learn
%pip install pandas

   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   --------------- ------------------------ 3.1/8.0 MB 18.4 MB/s eta 0:00:01
   ----------------------------------- ---- 7.1/8.0 MB 18.2 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 13.4 MB/s  0:00:00
   ---------------------------------------- 0.0/36.3 MB ? eta -:--:--
   - -------------------------------------- 1.6/36.3 MB 16.8 MB/s eta 0:00:03
   --- ------------------------------------ 3.4/36.3 MB 8.4 MB/s eta 0:00:04
   ----- ---------------------------------- 5.2/36.3 MB 8.6 MB/s eta 0:00:04
   ------ --------------------------------- 5.8/36.3 MB 7.8 MB/s eta 0:00:04
   ------ --------------------------------- 6.3/36.3 MB 6.5 MB/s eta 0:00:05
   -------- ------------------------------- 7.6/36.3 MB 6.1 MB/s eta 0:00:05
   ----------- ---------------------------- 10.5/36.3 MB 7.2 MB/s eta 0:00:04
   -------------- ------------------------- 13.1/36.3 MB 8.1 MB/s eta 0:00:03
   -----------

In [1]:
import cv2
import mediapipe as mp
import time
import csv
import os

In [9]:
# 1. Setup MediaPipe Tasks API
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Initialize the model options (Make sure 'hand_landmarker.task' is in your folder!)
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='hand_landmarker.task'),
    running_mode=VisionRunningMode.VIDEO,
    num_hands=1,
    min_hand_detection_confidence=0.7)

landmarker = HandLandmarker.create_from_options(options)

# 2. Setup ASL Alphabet Labels & Collection Variables
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'I Love You']  # Add the rest of the alphabet here
number_samples = 100       
DATA_FILE = 'asl_landmarks_person1.csv'

# 3. Create CSV and write headers (x0, y0, z0, x1, y1, z1... up to 20)
if not os.path.exists(DATA_FILE):
    with open(DATA_FILE, mode='w', newline='') as f:
        writer = csv.writer(f)
        headers = ['label']
        for i in range(21):
            headers.extend([f'x{i}', f'y{i}', f'z{i}'])
        writer.writerow(headers)

# 4. Start Webcam Loop
cap = cv2.VideoCapture(0)
start_time = time.time() # Used to calculate video timestamps

for label in labels:
    print(f'Get ready to sign: {label}')
    cv2.waitKey(5000) 
    
    samples_collected = 0
    while samples_collected < number_samples:
        ret, frame = cap.read()
        if not ret:
            break
            
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Convert the frame to MediaPipe's specialized Image object
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
        
        # Calculate strictly increasing timestamp in milliseconds
        timestamp_ms = int((time.time() - start_time) * 1000)
        
        # Process the frame
        results = landmarker.detect_for_video(mp_image, timestamp_ms)
        
        # If a hand is detected
        if results.hand_landmarks:
            # We only configured it for 1 hand, so we grab the first detected hand
            hand = results.hand_landmarks[0]
            
            # Draw green circles on the joints using standard OpenCV
            for lm in hand:
                x_px, y_px = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                cv2.circle(frame, (x_px, y_px), 5, (0, 255, 0), -1)
                
            # Flatten the 21 (x,y,z) coordinates into a single list
            landmark_row = [label]
            for lm in hand:
                landmark_row.extend([lm.x, lm.y, lm.z])
            
            # Append the row to our CSV file
            with open(DATA_FILE, mode='a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(landmark_row)
                
            samples_collected += 1
            time.sleep(0.1) 
                
        # Show the live feed
        cv2.putText(frame, f'Collecting "{label}": {samples_collected}/{number_samples}', 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('ASL Data Collection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
print("Data collection complete! Your CSV is ready.")

Get ready to sign: A
Get ready to sign: B
Get ready to sign: C
Get ready to sign: D
Get ready to sign: E
Get ready to sign: F
Get ready to sign: G
Get ready to sign: H
Get ready to sign: I
Get ready to sign: K
Get ready to sign: L
Get ready to sign: M
Get ready to sign: N
Get ready to sign: O
Get ready to sign: P
Get ready to sign: Q
Get ready to sign: R
Get ready to sign: S
Get ready to sign: T
Get ready to sign: U
Get ready to sign: V
Get ready to sign: W
Get ready to sign: X
Get ready to sign: Y
Get ready to sign: I Love You
Data collection complete! Your CSV is ready.
